In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install --upgrade pip
!pip install farm-haystack[colab,inference]

In [3]:
import pandas as pd
from haystack.document_stores import InMemoryDocumentStore, FAISSDocumentStore, ElasticsearchDocumentStore
from haystack.nodes import EmbeddingRetriever, DensePassageRetriever
from haystack import Pipeline
from haystack.pipelines import DocumentSearchPipeline
from haystack.nodes import BM25Retriever, SentenceTransformersRanker, DenseRetriever, DensePassageRetriever
from haystack.utils import print_documents, print_answers, print_questions


In [12]:
df = pd.read_csv('/content/gdrive/MyDrive/advices_assignment/data/cleaned_advices.csv')

In [13]:
df.head()

,advice,cleaned_advice,count
0,"['Drink plenty of water', 'eat hot food']",drink plenty of eat hot food,6
1,"['avoid oily and spicy foods', 'take steam thr...",avoid oily and spicy take steam thrice a day f...,19
2,"['eat hot food', 'normal regular diet ', 'all ...",eat hot normal regular diet all foods can be u...,29
3,"['Take Rest', 'Drink plenty of water', 'monito...",take drink plenty of monitor steam inhalation,7
4,"['Steam inhalation 4x a day', 'Monitor BP, SPO...",steam inhalation a monitor spo2 and temperatur...,53


In [ ]:
document_store = InMemoryDocumentStore(similarity='dot_product', use_bm25=False)
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    # embedding_model='medicalai/ClinicalBERT',
    use_gpu=True,
    scale_score=False,
)




In [ ]:
# We want to match "incoming query" <-> "stored advice".
advices = list(df["cleaned_advice"].values)
df["embedding"] = retriever.embed_queries(queries=advices).tolist()
df = df.rename(columns={"cleaned_advice": "content"})
df = df[['content', 'advice', 'embedding']]
# Create embeddings for all the advices from the cleaned advices
# Convert Dataframe to list of dicts and index them in our DocumentStore
docs_to_index = df.to_dict(orient="records")
document_store.write_documents(docs_to_index)

In [17]:
pipe = DocumentSearchPipeline(retriever)
result = pipe.run('I think I have fever ', )
print_documents(result, print_name=False, print_meta=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: I think I have fever 

{   'content': 'review if no fever appears',
    'meta': {'advice': "['Review if  no improvement/ fever appears ']"}}

{   'content': 'take soft drink electral glass drink plenty of take get fever '
               'profile test done',
    'meta': {   'advice': "['Take soft diet', 'Drink electral 1 glass daily', "
                          "'Drink plenty of water', 'Take Rest', 'Get fever "
                          "profile test done']"}}

{   'content': 'complete blood urine routine and drink plenty of fever panel '
               'test',
    'meta': {   'advice': "['Complete blood count\\n\\nCBC\\n', 'Urine Routine "
                          "and microscopic', 'Drink plenty of water', 'Fever "
                          "Panel test']"}}

{   'content': 'refer to fever clinic civil hospital',
    'meta': {'advice': "['REFER TO FEVER CLINIC CIVIL HOSPITAL']"}}

{   'content': 'no travel till fever subsides continuously for two days take '
               '

In [24]:
res_df = pd.DataFrame(result['documents'])[['content', 'score', 'meta']]

In [30]:
res_df

,content,score,meta
0,review if no fever appears,0.719490,{'advice': '['Review if no improvement/ fever...
1,take soft drink electral glass drink plenty of...,0.685355,"{'advice': '['Take soft diet', 'Drink electral..."
2,complete blood urine routine and drink plenty ...,0.669295,"{'advice': '['Complete blood count\n\nCBC\n', ..."
3,refer to fever clinic civil hospital,0.640512,{'advice': '['REFER TO FEVER CLINIC CIVIL HOSP...
4,no travel till fever subsides continuously for...,0.616619,{'advice': '['No travel till fever subsides co...
5,review if monitor fever,0.608598,"{'advice': '['REVIEW IF CONCERNS', 'MONITOR FE..."
6,report to local hospital or healthcare centre ...,0.568139,{'advice': '['Report to local hospital or heal...
7,monitor spo2 and temperature every consult a p...,0.544122,"{'advice': '['Monitor BP, SPO2 and Temperature..."
8,ice sponging on forehead if fever come rest fo...,0.540275,"{'advice': '[""Ice sponging on forehead if feve..."
9,maintain steam inhalation a lie in prone posit...,0.538817,"{'advice': '['Maintain temperature,spo2, BP ch..."
